# F1 Aerodynamics: Complex Flow Phenomena Exploration

This notebook explores various flow regimes to **identify and visualize complex phenomena** such as:
- Vortex formation and shedding patterns
- Turbulent wake structures
- Boundary layer separation
- Ground effect interaction
- Flow regime transitions

Each simulation captures the full temporal evolution to reveal transient and chaotic behavior.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import numpy as np
from lbm_core import LBMSolver
from boundaries import TunnelBoundaries
from analysis import plot_complexity_dashboard
from aerodynamics import calculate_lift_drag, check_ground_effect
import json
from datetime import datetime
from pathlib import Path
import pickle

## Configuration: Parameter Space for Phenomena Discovery

In [2]:
# --- BASE CONFIGURATION ---
NX, NY = 500, 200
GROUND_TYPE = "no_slip"
SIMULATION_STEPS = 3000
CHECKPOINT_INTERVAL = 100  # Save snapshots frequently to capture evolution

# --- PARAMETER SWEEP FOR PHENOMENA ---
# Different ride heights may reveal different flow regimes
RIDE_HEIGHTS = [10, 19, 35]  # Low (strong ground effect), Medium, High (weak ground effect)

# Reynolds numbers spanning laminar → transitional → turbulent
REYNOLDS_NUMBERS = [1000, 5000, 10000, 20000]  # Different complexity regimes

# --- OUTPUT CONFIGURATION ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = Path(f"flow_phenomena_{timestamp}")
results_dir.mkdir(exist_ok=True)

(results_dir / "snapshots").mkdir(exist_ok=True)      # Flow field evolution
(results_dir / "animations").mkdir(exist_ok=True)     # Animated visualizations
(results_dir / "vorticity").mkdir(exist_ok=True)      # Vorticity fields
(results_dir / "timeseries").mkdir(exist_ok=True)     # Force/flow data over time
(results_dir / "analysis").mkdir(exist_ok=True)       # Complexity metrics

print(f"Phenomena exploration results → {results_dir}")
print(f"Total configurations: {len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)}")
print(f"\nExploring flow regimes:")
print(f"  Ride heights: {RIDE_HEIGHTS}")
print(f"  Reynolds numbers: {REYNOLDS_NUMBERS}")

Phenomena exploration results → flow_phenomena_20260126_173315
Total configurations: 12

Exploring flow regimes:
  Ride heights: [10, 19, 35]
  Reynolds numbers: [1000, 5000, 10000, 20000]


## Phenomena Tracking System

In [3]:
class FlowPhenomenaTracker:
    """Track and analyze complex flow phenomena"""
    
    def __init__(self, ride_height, reynolds, nx, ny):
        self.ride_height = ride_height
        self.reynolds = reynolds
        self.nx = nx
        self.ny = ny
        self.timestamp = datetime.now().isoformat()
        
        # Evolution tracking
        self.velocity_snapshots = []      # Full flow fields at intervals
        self.vorticity_snapshots = []     # Vorticity fields
        self.snapshot_steps = []          # Timesteps of snapshots
        
        # Time series data
        self.lift_history = []
        self.drag_history = []
        self.step_history = []
        self.max_velocity_history = []    # Track peak velocities
        self.vorticity_magnitude = []     # Total vorticity in domain
        
        # Phenomena indicators
        self.vortex_shedding_detected = False
        self.shedding_frequency = None
        self.separation_points = []
        self.turbulence_intensity = None
        
    def add_snapshot(self, step, velocity_field, bounds_mask):
        """Capture full flow state for later analysis"""
        self.snapshot_steps.append(step)
        
        # Store velocity field
        v_mag = np.sqrt(velocity_field[:,:,0]**2 + velocity_field[:,:,1]**2)
        v_mag[bounds_mask] = np.nan
        self.velocity_snapshots.append(v_mag.copy())
        
        # Calculate and store vorticity (curl of velocity)
        vorticity = self._calculate_vorticity(velocity_field, bounds_mask)
        self.vorticity_snapshots.append(vorticity)
        self.vorticity_magnitude.append(np.nansum(np.abs(vorticity)))
    
    def _calculate_vorticity(self, velocity_field, mask):
        """Calculate vorticity field (ω = ∂v/∂x - ∂u/∂y)"""
        u = velocity_field[:,:,0]
        v = velocity_field[:,:,1]
        
        # Compute gradients
        du_dy = np.gradient(u, axis=0)
        dv_dx = np.gradient(v, axis=1)
        
        vorticity = dv_dx - du_dy
        vorticity[mask] = np.nan
        
        return vorticity
    
    def add_step_data(self, step, lift, drag, max_vel):
        """Record time series data"""
        self.step_history.append(step)
        self.lift_history.append(lift)
        self.drag_history.append(drag)
        self.max_velocity_history.append(max_vel)
    
    def analyze_phenomena(self):
        """Detect and characterize flow phenomena"""
        # Detect vortex shedding via force oscillations
        if len(self.lift_history) > 500:
            lift_signal = np.array(self.lift_history[-1000:])
            lift_signal = lift_signal - np.mean(lift_signal)
            
            # FFT to find dominant frequency
            fft = np.fft.fft(lift_signal)
            freqs = np.fft.fftfreq(len(lift_signal))
            power = np.abs(fft)**2
            
            # Find peak frequency (excluding DC)
            positive_freqs = freqs[1:len(freqs)//2]
            positive_power = power[1:len(power)//2]
            
            if len(positive_power) > 0:
                peak_idx = np.argmax(positive_power)
                peak_freq = positive_freqs[peak_idx]
                peak_power = positive_power[peak_idx]
                
                # If peak is significant, we have periodic shedding
                if peak_power > 5 * np.mean(positive_power):
                    self.vortex_shedding_detected = True
                    self.shedding_frequency = abs(peak_freq)
        
        # Calculate turbulence intensity from velocity fluctuations
        if len(self.max_velocity_history) > 100:
            vel_array = np.array(self.max_velocity_history[-500:])
            mean_vel = np.mean(vel_array)
            if mean_vel > 0:
                self.turbulence_intensity = np.std(vel_array) / mean_vel
    
    def get_phenomena_summary(self):
        """Return dictionary of detected phenomena"""
        return {
            'ride_height': self.ride_height,
            'reynolds': self.reynolds,
            'vortex_shedding': self.vortex_shedding_detected,
            'shedding_frequency': float(self.shedding_frequency) if self.shedding_frequency else None,
            'turbulence_intensity': float(self.turbulence_intensity) if self.turbulence_intensity else None,
            'mean_vorticity': float(np.mean(self.vorticity_magnitude)) if self.vorticity_magnitude else None,
            'peak_vorticity': float(np.max(self.vorticity_magnitude)) if self.vorticity_magnitude else None,
        }
    
    def save(self, base_dir):
        """Save all phenomena data"""
        base_dir = Path(base_dir)
        prefix = f"rh{self.ride_height}_re{self.reynolds}"
        
        # Save phenomena summary
        with open(base_dir / "analysis" / f"{prefix}_phenomena.json", 'w') as f:
            json.dump(self.get_phenomena_summary(), f, indent=2)
        
        # Save time series
        np.savez_compressed(
            base_dir / "timeseries" / f"{prefix}_evolution.npz",
            steps=self.step_history,
            lift=self.lift_history,
            drag=self.drag_history,
            max_velocity=self.max_velocity_history,
            vorticity_magnitude=self.vorticity_magnitude
        )
        
        # Save snapshots
        np.savez_compressed(
            base_dir / "snapshots" / f"{prefix}_snapshots.npz",
            velocity_fields=np.array(self.velocity_snapshots),
            vorticity_fields=np.array(self.vorticity_snapshots),
            snapshot_steps=self.snapshot_steps
        )
        
        print(f"  💾 Saved phenomena data: {prefix}")
        return prefix


def create_geometry(bounds, ride_height):
    """Create F1 car geometry"""
    bounds.add_ground(type=GROUND_TYPE)
    
    bounds.add_f1_wing_proxy(
        x_pos=150,
        height=ride_height,
        length=60,
        slope=0.45
    )
    
    bounds.add_rectangular_obstacle(
        x_start=210,
        y_start=ride_height + 1,
        length=120,
        height=20
    )
    
    bounds.add_reverse_triangle(
        x_pos=240,
        height=ride_height + 20,
        length=90,
        slope=2/9
    )
    
    bounds.add_f1_wing_proxy(
        x_pos=300,
        height=ride_height + 30,
        length=30,
        slope=0.2
    )

## Run Phenomena Exploration

In [4]:
all_trackers = []

total_sims = len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)
sim_counter = 0

print("="*70)
print("EXPLORING FLOW PHENOMENA")
print("="*70)

for reynolds in REYNOLDS_NUMBERS:
    for ride_height in RIDE_HEIGHTS:
        sim_counter += 1
        print(f"\n[{sim_counter}/{total_sims}] Re={reynolds}, Ride Height={ride_height}")
        print("-" * 70)
        
        tracker = FlowPhenomenaTracker(ride_height, reynolds, NX, NY)
        
        solver = LBMSolver(NX, NY, reynolds, u_inlet=0.1)
        bounds = TunnelBoundaries(NX, NY)
        create_geometry(bounds, ride_height)
        
        relevant_x_start = 145
        relevant_x_end = 335
        relevant_y_start = 5
        relevant_y_end = min(75, ride_height + 60)
        
        print(f"  Running simulation to capture flow evolution...")
        
        for step in range(SIMULATION_STEPS):
            solver.collide_and_stream(bounds.mask)
            bounds.apply_inlet_outlet(solver)
            
            # Always track forces and max velocity
            if step % 10 == 0:
                fx, fy = calculate_lift_drag(
                    solver, bounds,
                    x_start=relevant_x_start,
                    x_end=relevant_x_end,
                    y_start=relevant_y_start,
                    y_end=relevant_y_end
                )
                max_vel = np.max(np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2))
                tracker.add_step_data(step, fy, fx, max_vel)
            
            # Capture flow snapshots
            if step % CHECKPOINT_INTERVAL == 0 or step == SIMULATION_STEPS - 1:
                tracker.add_snapshot(step, solver.u, bounds.mask)
                
                if step % (CHECKPOINT_INTERVAL * 5) == 0:
                    print(f"    Step {step}/{SIMULATION_STEPS} - Captured snapshot")
        
        # Analyze what phenomena occurred
        tracker.analyze_phenomena()
        summary = tracker.get_phenomena_summary()
        
        print(f"\n  🔍 PHENOMENA DETECTED:")
        print(f"     Vortex Shedding: {'YES' if summary['vortex_shedding'] else 'NO'}")
        if summary['shedding_frequency']:
            print(f"     Shedding Frequency: {summary['shedding_frequency']:.6f}")
        if summary['turbulence_intensity']:
            print(f"     Turbulence Intensity: {summary['turbulence_intensity']:.4f}")
        print(f"     Mean Vorticity: {summary['mean_vorticity']:.4f}")
        
        tracker.save(results_dir)
        all_trackers.append(tracker)

print("\n" + "="*70)
print("PHENOMENA EXPLORATION COMPLETE")
print("="*70)

EXPLORING FLOW PHENOMENA

[1/12] Re=1000, Ride Height=10
----------------------------------------------------------------------
  Running simulation to capture flow evolution...
    Step 0/3000 - Captured snapshot
    Step 500/3000 - Captured snapshot
    Step 1000/3000 - Captured snapshot
    Step 1500/3000 - Captured snapshot
    Step 2000/3000 - Captured snapshot
    Step 2500/3000 - Captured snapshot

  🔍 PHENOMENA DETECTED:
     Vortex Shedding: NO
     Turbulence Intensity: 0.1083
     Mean Vorticity: 152.4885
  💾 Saved phenomena data: rh10_re1000

[2/12] Re=1000, Ride Height=19
----------------------------------------------------------------------
  Running simulation to capture flow evolution...
    Step 0/3000 - Captured snapshot
    Step 500/3000 - Captured snapshot
    Step 1000/3000 - Captured snapshot
    Step 1500/3000 - Captured snapshot
    Step 2000/3000 - Captured snapshot
    Step 2500/3000 - Captured snapshot

  🔍 PHENOMENA DETECTED:
     Vortex Shedding: NO
     Tu

## Visualize Flow Structures: Vorticity Fields

In [5]:
print("Creating vorticity visualizations...\n")

for tracker in all_trackers:
    # Select key snapshots to visualize
    n_snapshots = len(tracker.vorticity_snapshots)
    if n_snapshots == 0:
        continue
    
    # Show: initial, early development, mid, late
    indices = [0, n_snapshots//4, n_snapshots//2, -1]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()
    
    # Find global vorticity range for consistent colormap
    all_vort = np.concatenate([v.flatten() for v in tracker.vorticity_snapshots])
    vmax = np.nanpercentile(np.abs(all_vort), 99)
    
    for i, idx in enumerate(indices):
        ax = axes[i]
        vorticity = tracker.vorticity_snapshots[idx]
        step = tracker.snapshot_steps[idx]
        
        im = ax.imshow(vorticity, origin='lower', cmap='RdBu_r', 
                      vmin=-vmax, vmax=vmax, interpolation='bilinear')
        ax.set_title(f'Step {step}\nVorticity Field', fontsize=12, fontweight='bold')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        plt.colorbar(im, ax=ax, label='Vorticity (ω)')
    
    fig.suptitle(f'Vortex Evolution - Re={tracker.reynolds}, Ride Height={tracker.ride_height}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    filename = f"rh{tracker.ride_height}_re{tracker.reynolds}_vorticity_evolution.png"
    plt.savefig(results_dir / "vorticity" / filename, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"  📊 {filename}")

print("\nVorticity visualizations complete!")

Creating vorticity visualizations...

  📊 rh10_re1000_vorticity_evolution.png
  📊 rh19_re1000_vorticity_evolution.png
  📊 rh35_re1000_vorticity_evolution.png
  📊 rh10_re5000_vorticity_evolution.png
  📊 rh19_re5000_vorticity_evolution.png
  📊 rh35_re5000_vorticity_evolution.png
  📊 rh10_re10000_vorticity_evolution.png
  📊 rh19_re10000_vorticity_evolution.png
  📊 rh35_re10000_vorticity_evolution.png
  📊 rh10_re20000_vorticity_evolution.png
  📊 rh19_re20000_vorticity_evolution.png
  📊 rh35_re20000_vorticity_evolution.png

Vorticity visualizations complete!


## Compare Phenomena Across Parameters

In [6]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Organize data
re_values = sorted(list(set(t.reynolds for t in all_trackers)))
rh_values = sorted(list(set(t.ride_height for t in all_trackers)))

# Plot 1: Turbulence Intensity Heatmap
ax = axes[0, 0]
ti_matrix = np.zeros((len(re_values), len(rh_values)))
for i, re in enumerate(re_values):
    for j, rh in enumerate(rh_values):
        tracker = next((t for t in all_trackers if t.reynolds == re and t.ride_height == rh), None)
        if tracker and tracker.turbulence_intensity:
            ti_matrix[i, j] = tracker.turbulence_intensity

im = ax.imshow(ti_matrix, aspect='auto', cmap='plasma', origin='lower')
ax.set_xticks(range(len(rh_values)))
ax.set_yticks(range(len(re_values)))
ax.set_xticklabels(rh_values)
ax.set_yticklabels(re_values)
ax.set_xlabel('Ride Height', fontsize=12)
ax.set_ylabel('Reynolds Number', fontsize=12)
ax.set_title('Turbulence Intensity', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Turbulence Intensity')

# Plot 2: Vorticity Magnitude
ax = axes[0, 1]
vort_matrix = np.zeros((len(re_values), len(rh_values)))
for i, re in enumerate(re_values):
    for j, rh in enumerate(rh_values):
        tracker = next((t for t in all_trackers if t.reynolds == re and t.ride_height == rh), None)
        if tracker and tracker.vorticity_magnitude:
            vort_matrix[i, j] = np.mean(tracker.vorticity_magnitude)

im = ax.imshow(vort_matrix, aspect='auto', cmap='viridis', origin='lower')
ax.set_xticks(range(len(rh_values)))
ax.set_yticks(range(len(re_values)))
ax.set_xticklabels(rh_values)
ax.set_yticklabels(re_values)
ax.set_xlabel('Ride Height', fontsize=12)
ax.set_ylabel('Reynolds Number', fontsize=12)
ax.set_title('Mean Vorticity Magnitude', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Vorticity')

# Plot 3: Force Oscillation Examples (most complex case)
ax = axes[1, 0]
most_complex = max(all_trackers, key=lambda t: t.turbulence_intensity if t.turbulence_intensity else 0)
ax.plot(most_complex.step_history, most_complex.lift_history, 'b-', linewidth=1, alpha=0.7, label='Lift')
ax.plot(most_complex.step_history, most_complex.drag_history, 'r-', linewidth=1, alpha=0.7, label='Drag')
ax.set_xlabel('Simulation Step', fontsize=12)
ax.set_ylabel('Force', fontsize=12)
ax.set_title(f'Most Complex Flow\n(Re={most_complex.reynolds}, RH={most_complex.ride_height})', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Vortex Shedding Detection Summary
ax = axes[1, 1]
shedding_matrix = np.zeros((len(re_values), len(rh_values)))
for i, re in enumerate(re_values):
    for j, rh in enumerate(rh_values):
        tracker = next((t for t in all_trackers if t.reynolds == re and t.ride_height == rh), None)
        if tracker:
            shedding_matrix[i, j] = 1 if tracker.vortex_shedding_detected else 0

im = ax.imshow(shedding_matrix, aspect='auto', cmap='RdYlGn', origin='lower', vmin=0, vmax=1)
ax.set_xticks(range(len(rh_values)))
ax.set_yticks(range(len(re_values)))
ax.set_xticklabels(rh_values)
ax.set_yticklabels(re_values)
ax.set_xlabel('Ride Height', fontsize=12)
ax.set_ylabel('Reynolds Number', fontsize=12)
ax.set_title('Vortex Shedding Detected\n(Green = Yes, Red = No)', fontsize=14, fontweight='bold')

# Add text annotations
for i in range(len(re_values)):
    for j in range(len(rh_values)):
        text = ax.text(j, i, 'YES' if shedding_matrix[i, j] else 'NO',
                      ha="center", va="center", color="black", fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(results_dir / "analysis" / "phenomena_comparison.png", dpi=300, bbox_inches='tight')
plt.close()

print("\n📈 Created phenomena comparison plots")


📈 Created phenomena comparison plots


## Generate Detailed Visualizations for Each Configuration

In [7]:
print("Creating detailed multi-panel visualizations...\n")

for tracker in all_trackers:
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Row 1: Flow evolution (velocity magnitude)
    steps_to_show = [0, len(tracker.velocity_snapshots)//2, -1]
    for i, idx in enumerate(steps_to_show):
        ax = fig.add_subplot(gs[0, i])
        vel = tracker.velocity_snapshots[idx]
        step = tracker.snapshot_steps[idx]
        
        im = ax.imshow(vel, origin='lower', cmap='magma', vmin=0, vmax=0.15)
        ax.set_title(f'Velocity - Step {step}', fontweight='bold')
        plt.colorbar(im, ax=ax, label='|u|')
    
    # Row 2: Vorticity evolution
    all_vort = np.concatenate([v.flatten() for v in tracker.vorticity_snapshots])
    vmax = np.nanpercentile(np.abs(all_vort), 99)
    
    for i, idx in enumerate(steps_to_show):
        ax = fig.add_subplot(gs[1, i])
        vort = tracker.vorticity_snapshots[idx]
        step = tracker.snapshot_steps[idx]
        
        im = ax.imshow(vort, origin='lower', cmap='RdBu_r', 
                      vmin=-vmax, vmax=vmax, interpolation='bilinear')
        ax.set_title(f'Vorticity - Step {step}', fontweight='bold')
        plt.colorbar(im, ax=ax, label='ω')
    
    # Row 3: Time series analysis
    # Force history
    ax = fig.add_subplot(gs[2, 0])
    ax.plot(tracker.step_history, tracker.lift_history, 'b-', linewidth=1, alpha=0.8, label='Lift')
    ax.plot(tracker.step_history, tracker.drag_history, 'r-', linewidth=1, alpha=0.8, label='Drag')
    ax.set_xlabel('Step')
    ax.set_ylabel('Force')
    ax.set_title('Aerodynamic Forces', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Vorticity evolution
    ax = fig.add_subplot(gs[2, 1])
    ax.plot(tracker.snapshot_steps, tracker.vorticity_magnitude, 'g-', linewidth=2)
    ax.set_xlabel('Step')
    ax.set_ylabel('Total |Vorticity|')
    ax.set_title('Vorticity Magnitude Evolution', fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Velocity statistics
    ax = fig.add_subplot(gs[2, 2])
    ax.plot(tracker.step_history, tracker.max_velocity_history, 'purple', linewidth=2)
    ax.set_xlabel('Step')
    ax.set_ylabel('Max Velocity')
    ax.set_title('Peak Flow Velocity', fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Overall title with phenomena summary
    phenomena = "Vortex Shedding" if tracker.vortex_shedding_detected else "No Shedding"
    ti = f"TI={tracker.turbulence_intensity:.3f}" if tracker.turbulence_intensity else "TI=N/A"
    fig.suptitle(f'Flow Phenomena Analysis\nRe={tracker.reynolds}, Ride Height={tracker.ride_height}\n{phenomena}, {ti}',
                 fontsize=16, fontweight='bold')
    
    filename = f"rh{tracker.ride_height}_re{tracker.reynolds}_detailed_analysis.png"
    plt.savefig(results_dir / "analysis" / filename, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"  📊 {filename}")

print("\nDetailed visualizations complete!")

Creating detailed multi-panel visualizations...

  📊 rh10_re1000_detailed_analysis.png
  📊 rh19_re1000_detailed_analysis.png
  📊 rh35_re1000_detailed_analysis.png
  📊 rh10_re5000_detailed_analysis.png
  📊 rh19_re5000_detailed_analysis.png
  📊 rh35_re5000_detailed_analysis.png
  📊 rh10_re10000_detailed_analysis.png
  📊 rh19_re10000_detailed_analysis.png
  📊 rh35_re10000_detailed_analysis.png
  📊 rh10_re20000_detailed_analysis.png
  📊 rh19_re20000_detailed_analysis.png
  📊 rh35_re20000_detailed_analysis.png

Detailed visualizations complete!


## Summary: Identified Phenomena

In [8]:
# Compile all phenomena
phenomena_list = [t.get_phenomena_summary() for t in all_trackers]

# Save master summary
with open(results_dir / "phenomena_summary.json", 'w') as f:
    json.dump(phenomena_list, f, indent=2)

print("="*70)
print("PHENOMENA DISCOVERY SUMMARY")
print("="*70)
print(f"\nTotal configurations explored: {len(all_trackers)}")

shedding_cases = [p for p in phenomena_list if p['vortex_shedding']]
print(f"\nVortex shedding detected in {len(shedding_cases)}/{len(phenomena_list)} cases:")
for p in shedding_cases:
    print(f"  • Re={p['reynolds']}, RH={p['ride_height']}, f={p['shedding_frequency']:.6f}")

if phenomena_list:
    avg_ti = np.mean([p['turbulence_intensity'] for p in phenomena_list if p['turbulence_intensity']])
    max_ti = max([p['turbulence_intensity'] for p in phenomena_list if p['turbulence_intensity']])
    print(f"\nTurbulence Intensity:")
    print(f"  • Average: {avg_ti:.4f}")
    print(f"  • Maximum: {max_ti:.4f}")
    
    most_turbulent = max(phenomena_list, key=lambda p: p['turbulence_intensity'] if p['turbulence_intensity'] else 0)
    print(f"  • Most turbulent case: Re={most_turbulent['reynolds']}, RH={most_turbulent['ride_height']}")

print(f"\n📁 All results saved to: {results_dir}")
print(f"\n📊 Generated outputs:")
print(f"  • {len(list((results_dir / 'snapshots').glob('*.npz')))} flow evolution datasets")
print(f"  • {len(list((results_dir / 'vorticity').glob('*.png')))} vorticity visualizations")
print(f"  • {len(list((results_dir / 'analysis').glob('*.png')))} analysis plots")
print(f"  • {len(list((results_dir / 'analysis').glob('*.json')))} phenomena reports")

PHENOMENA DISCOVERY SUMMARY

Total configurations explored: 12

Vortex shedding detected in 0/12 cases:

Turbulence Intensity:
  • Average: 0.1037
  • Maximum: 0.1125
  • Most turbulent case: Re=20000, RH=35

📁 All results saved to: flow_phenomena_20260126_173315

📊 Generated outputs:
  • 12 flow evolution datasets
  • 12 vorticity visualizations
  • 13 analysis plots
  • 12 phenomena reports
